In [1]:
import os

import django
from django.core.management import call_command

# Simulate manage.py
os.environ["DJANGO_SETTINGS_MODULE"] = "arches_querysets.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

call_command("add_test_data")

Creating test data...
Finished!


In [ ]:
from pprint import pprint

from arches_querysets.models import *

resource = ResourceTileTree.get_tiles(graph_slug="datatype_lookups").exclude(url=None).get()
pprint(resource.aliased_data)
pprint(resource.aliased_data.datatypes_1.aliased_data)

In [ ]:
resource = ResourceTileTree.get_tiles(
    graph_slug="datatype_lookups", as_representation=True
).exclude(url=None).get()
pprint(resource.aliased_data)
pprint(resource.aliased_data.datatypes_1.aliased_data)

In [ ]:
test_resources = ResourceTileTree.get_tiles("datatype_lookups")
# This "shallow" lookup works whether or not the node is nested (todo: add test case)
resource = test_resources.filter(string__any_lang_contains="forty-two").first()
resource.aliased_data.datatypes_1.aliased_data.string

In [ ]:
# You can save back with or without the display value / interchange_value wrapper
resource.aliased_data.datatypes_1.aliased_data.string = 'forty-three'
resource.save()
# But without a request, we assume anonymous user, and it ends up in provisional edits
resource.aliased_data.datatypes_1.provisionaledits

In [ ]:
resource.aliased_data.datatypes_1.aliased_data.string = 'forty-three'
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string